# Topic modelling for InnerSpeech dataset (Japanese)


Author : Romy Beauté\
Date created : 02/12/2024\
Last modified : 25/02/2025\
Corresp : r.beaut@sussex.ac.uk

Selection of sentence transformer embedding models :
https://www.sbert.net/docs/pretrained_models.html

The all-mpnet-base-v2 model provides the best quality, while all-MiniLM-L6-v2 is 5 times faster and still offers good quality



In [1]:
# %%capture
# !pip install bertopic accelerate bitsandbytes xformers adjustText
# !pip install llama-cpp-python
# !{sys.executable} -m pip install "scipy==1.9.0" "scikit-image==0.23.2"

import os
import pickle
import nltk
import sys
import pandas as pd
import numpy as np
from bertopic import BERTopic
from bertopic.backend import languages

os.environ["TOKENIZERS_PARALLELISM"] = "True"
nltk.download('stopwords')

script_dir = os.path.dirname(os.path.abspath('__file__'))
print("Script directory:", script_dir)
multilingual_dir = os.path.join(script_dir, 'MULTILINGUAL')
sys.path.append(multilingual_dir)
print(f"Added {multilingual_dir} to Python path")

from multiling_helpers import JapaneseProcessor, TopicModeler


# ------------------------------------------------------------------------------------------------
# SET PARAMS FOR DATASET
# ------------------------------------------------------------------------------------------------

project = "multilingual"
dataset = "innerspeech" 
language = "japanese"
#check that chosen language is supported by BERTopic internal language
if language not in languages:
    raise ValueError(f"Language '{language}' is not supported. Supported languages are: {languages}")

DATA_dir = os.path.join(script_dir, f'DATA/{project}/{language}/{dataset}')
print("DATA directory:", DATA_dir)

cache_dir = os.path.join(DATA_dir, "cache")
print("Cache directory:", cache_dir)
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)


save_reports = True


Script directory: /Users/rbeaute/Projects/MOSAIC
Added /Users/rbeaute/Projects/MOSAIC/MULTILINGUAL to Python path
DATA directory: /Users/rbeaute/Projects/MOSAIC/DATA/multilingual/japanese/innerspeech
Cache directory: /Users/rbeaute/Projects/MOSAIC/DATA/multilingual/japanese/innerspeech/cache


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rbeaute/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

reports_path = os.path.join(DATA_dir,f"{dataset}.xlsx")
df = pd.read_excel(reports_path)

# ------------------------------------------------------------------------------------------------
colnames = pd.read_excel(reports_path).columns.tolist()
print(len(colnames), "columns:", colnames)

non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).columns
print(f"Columns with non-numeric values (n={len(non_numeric_cols.tolist())}):", non_numeric_cols.tolist())

38 columns: ['タイムスタンプ', '性別（自認）を教えて下さい', '年齢を教えて下さい', '上の動画がどれだけ自分の内言として当てはまりますか？', '上の動画がどれだけ自分の内言として当てはまりますか？.1', '上の動画がどれだけ自分の内言として当てはまりますか？.2', '上の動画がどれだけ自分の内言として当てはまりますか？.3', '上の動画がどれだけ自分の内言として当てはまりますか？.4', '上の動画がどれだけ自分の内言として当てはまりますか？.5', '上の動画がどれだけ自分の内言として当てはまりますか？.6', '上記とは異なる内言で思考している場合は教えて下さい（任意）', '上の動画がどれだけ自分の内言として当てはまりますか？.7', '上の動画がどれだけ自分の内言として当てはまりますか？.8', '上の動画がどれだけ自分の内言として当てはまりますか？.9', '上の動画がどれだけ自分の内言として当てはまりますか？.10', '上の動画がどれだけ自分の内言として当てはまりますか？.11', '上の動画がどれだけ自分の内言として当てはまりますか？.12', '上の動画がどれだけ自分の内言として当てはまりますか？.13', '上記とは異なる内言で思考している場合は教えて下さい（任意）.1', '上の動画がどれだけ自分の内言として当てはまりますか？.14', '上の動画がどれだけ自分の内言として当てはまりますか？.15', '上の動画がどれだけ自分の内言として当てはまりますか？.16', '上の動画がどれだけ自分の内言として当てはまりますか？.17', '上の動画がどれだけ自分の内言として当てはまりますか？.18', '上の動画がどれだけ自分の内言として当てはまりますか？.19', '上の動画がどれだけ自分の内言として当てはまりますか？.20', '上記とは異なる内言で思考している場合は教えて下さい（任意）.2', '内言の中に「視覚」は', '内言の中に「聴覚」は', '内言の中に「嗅覚」は', '内言の中に「味覚」は', '内言の中に「触覚」は', 'あなたの内言は，', 'メールを書くときに，頭の中だけで（メモなどを取らずに）事前に文面を組み立てることは可能ですか？', '粘土細工をつくるときに，頭の中だけで（スケッチなどをせず

non-numeric columns translated:

1. Timestamp
2. Gender (self-identified)
3. Age
4. If you think in different inner speech than above, please tell us (optional)
5. If you think in different inner speech than above, please tell us (optional).1
6. If you think in different inner speech than above, please tell us (optional).2
7. Please freely describe anything else about inner speech (optional)

### Isolate non-numeric variables (that might contain verbal reports)

In [3]:
# df of non-numeric columns
df_non_numeric = df[non_numeric_cols]
df_non_numeric.head(5)

,タイムスタンプ,性別（自認）を教えて下さい,年齢を教えて下さい,上記とは異なる内言で思考している場合は教えて下さい（任意）,上記とは異なる内言で思考している場合は教えて下さい（任意）.1,上記とは異なる内言で思考している場合は教えて下さい（任意）.2,内言について他に述べたいことがあれば自由に記述してください（任意）
0,2022-05-11 17:46:55.621,男性,35,NaN,NaN,NaN,NaN
1,2022-05-11 17:48:16.588,男性,26,NaN,NaN,NaN,NaN
2,2022-05-11 17:49:03.556,女性,21,NaN,NaN,NaN,NaN
3,2022-05-11 17:49:33.625,男性,34,NaN,NaN,NaN,NaN
4,2022-05-11 17:49:48.730,女性,53,NaN,NaN,NaN,NaN


non-numeric columns translated:

1. If you think in different inner speech than above, please tell us (optional)
2. If you think in different inner speech than above, please tell us (optional).1
3. If you think in different inner speech than above, please tell us (optional).2
4. Please freely describe anything else about inner speech (optional)


Participants were asked several questions in 3 different situations.
The situations were illustrated with videos: 
- A situation where you are looking for your wallet
- A situation where you are thinking about buying a car
- A situation where you are thinking about lunch
After they rated videos which explain different types of inner speech, the following question was asked:\
**Question: "If you are thinking in a different way, please write it down (optional)"**

In [4]:
#remove non_numeric_cols[0], non_numeric_cols[1], non_numeric_cols[2] from df_non_numeric
df_non_numeric = df_non_numeric.drop(columns=[non_numeric_cols[0], non_numeric_cols[1], non_numeric_cols[2]])


#remove all participants that have NaN in all columns from df_non_numeric
print(len(df_non_numeric)) #before removing NaNs
df_non_numeric = df_non_numeric.dropna(how='all')
print(len(df_non_numeric)) #after removing NaNs


# ------------------------------------------------------------------------------------------------
#update non_numeric_cols to reflect the removal of non_numeric_cols[0], non_numeric_cols[1], non_numeric_cols[2]
non_numeric_cols = df_non_numeric.columns
#remane text reports cols for clarity:
df_non_numeric = df_non_numeric.rename(columns={non_numeric_cols[0]: 'wallet_answer'})
df_non_numeric = df_non_numeric.rename(columns={non_numeric_cols[1]: 'car_answer'})
df_non_numeric = df_non_numeric.rename(columns={non_numeric_cols[2]: 'lunch_answer'})
df_non_numeric = df_non_numeric.rename(columns={non_numeric_cols[3]: 'reflection_answer'})

df_non_numeric.head(5)

5365
1313


,wallet_answer,car_answer,lunch_answer,reflection_answer
5,NaN,NaN,NaN,頭の中の独り言をこのような調査で改めて自覚することができ、また色々なパターンがあることを知り...
11,自分の動きを考えるときは、漫画のネームみたいな感じで思い浮かべる,NaN,NaN,NaN
14,NaN,NaN,味と匂いを想像し、それをいま自分が欲しているのかを自らに問う,他人の声が脳内でしている人がいるという話にすごく興味があるのですが、心理物理実験で音声のパラ...
24,基本は回答した通り映像なんですが、そこに自分が財布を置いた時、触った時の体の感覚も入ってる感じです,NaN,イメージと自分の言葉に加えて、食べたいかどうかを味を想像して考えてる気がします,NaN
26,NaN,NaN,音声と文字のどちらかだけではなく、組み合わさっていると思う。,頭の中では日本語で考えているという自覚はある（英語は勉強以外にほぼ使ったことはない）が、文字...


In [5]:
#list of reflection_answer, and remove NaN values
wallet_reports = df_non_numeric['wallet_answer'].dropna()
car_reports = df_non_numeric['car_answer'].dropna()
lunch_reports = df_non_numeric['lunch_answer'].dropna()
df_reports = df_non_numeric['reflection_answer'].dropna()

all_reports = pd.concat([wallet_reports, car_reports, lunch_reports, df_reports])
print(f"N = {len(all_reports)} reports in total, from N = {len(df_non_numeric)} participants")

N = 1979 reports in total, from N = 1313 participants


In [6]:
df_reports.iloc[0]

'頭の中の独り言をこのような調査で改めて自覚することができ、また色々なパターンがあることを知り面白かった。'

In [7]:
all_reflection_reports = all_reports.tolist() #all free text reports, including wallet, car, lunch, and reflection
reflection_reports = df_reports.tolist() #only reflection reports

print(f"N = {len(reflection_reports)} reflection reports")
print(f"N = {len(all_reflection_reports)} free text reports in total")

N = 731 reflection reports
N = 1979 free text reports in total


In [8]:


#save reflections reports to pickle file
reports_path = os.path.join(DATA_dir,f"{dataset}_reports.pkl")
with open(reports_path, 'wb') as f:
    pickle.dump(reflection_reports, f)
print(f"Saved raw reports to {reports_path}")


# save all reports to pickle file
reports_path = os.path.join(DATA_dir,f"{dataset}_all_reports.pkl")
with open(reports_path, 'wb') as f:
    pickle.dump(all_reflection_reports, f)
print(f"Saved all reports to {reports_path}")


Saved raw reports to /Users/rbeaute/Projects/MOSAIC/DATA/multilingual/japanese/innerspeech/innerspeech_reports.pkl
Saved all reports to /Users/rbeaute/Projects/MOSAIC/DATA/multilingual/japanese/innerspeech/innerspeech_all_reports.pkl


In [9]:
reflection_reports[:20]

['頭の中の独り言をこのような調査で改めて自覚することができ、また色々なパターンがあることを知り面白かった。',
 '他人の声が脳内でしている人がいるという話にすごく興味があるのですが、心理物理実験で音声のパラメータチューニングかA/Bテストかを繰り返すことでその人の脳内の声を具現化することってできますかね？（具現化しようとした途端に霞をつかむようにとらえどころのないものになってしまうなどの可能性もあるかもなあと思いつつ）',
 '頭の中では日本語で考えているという自覚はある（英語は勉強以外にほぼ使ったことはない）が、文字か音声かと言われると、どちらも混ざっている感じがする。\n映像や味などは思い出される事がある。',
 '自動思考というものなのか、直近で起きた失敗などを批判する考えが勝手に浮かんできたりすることがあるが、自分の声とか他人の声という認識はしていなかった。',
 '食べたいもの、欲しいものなどは、自分がそれを食べている、或いは使っているところを想像して決めているような感じでした。逆に、どこに置いたかな、というのは「自分が置いたときのイメージが湧かないからこそそういう状況になっている」ときは言葉で、「場面を辿ればわかる」ときは映像で思い浮かぶ感じでした。\n発達系の話になりますが視覚優位か聴覚優位かなどによっても変わりそうで面白いなと思いました。',
 '画像や映像が頭の中に流れ、その後に自分の声で考える事が多い。',
 'フローチャートや箇条書きで考えるのは便利そうなので意識的にやってみたい。',
 'キャスト全員が自分の声で話しているボイスドラマを聴いている風に内言します。',
 '思考中にふと反対意見・対立意見が他人の声で聞こえる',
 '言葉にできないことが多い。説明や言葉が足りず、内容を人に理解させることが難しい。',
 'この度は面白いテーマでのアンケートの機会をいただきありがとうございます。アンケートを通じて改めて、黙読している時も、文章を書いている時も必ず内言が生じていることに気づきました。アンケートの内容からそうではない人がいるのだということも知ることができ、大変面白くもありました。',
 '五感のどれをどのくらい使うかは、場合による。',
 'イメージだけを提示する別の自分と会話している感じはする',
 '全体的に「言われてみればこう